# Machine Learning lab 4: extending logistic regression
## Jake Rowland and Paul Herz
2017-10-01

## 1. Introduction

TODO

### 1.1 Background

TODO

### 1.2 Business Case

TODO

### 1.3 Serviceability

TODO

## 2. The dataset: preprocessing and review

### 2.1 Dataset preparation

TODO

### 2.2 Data quality

TODO

## 3. Something

